<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter

In [6]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [7]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [8]:
query_3_1 = f'''
      SELECT
              COUNT(id) AS "Количество вакансий"
        FROM  public.vacancies
'''

In [9]:
df = pd.read_sql_query(query_3_1, connection)
display(df)

,Количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [10]:
query_3_2 = f'''
      SELECT
              COUNT(id) AS "Количество работодателей"
        FROM  public.employers
'''

In [11]:
df = pd.read_sql_query(query_3_2, connection)
display(df)

,Количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [12]:
query_3_3 = f'''
      SELECT
              COUNT(id) AS "Количество регионов"
        FROM  public.areas
'''

In [13]:
df = pd.read_sql_query(query_3_3, connection)
display(df)

,Количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [14]:
query_3_4 = f'''
      SELECT
             COUNT(id) AS "Количество сфер деятельности"
        FROM  public.industries
'''

In [15]:
df = pd.read_sql_query(query_3_4, connection)
display(df)

,Количество сфер деятельности
0,294


***

### Вывод:
После проведения ознакомительного(предварительного) анализа имеющихся у нас таблиц, мы можем сделать некоторые выводы. Нам предоставлены данные о значительном количестве работодателей которым требуются специалисты с различным уровнем квалификации и опыта работы.
Также мы понимаем, что в настоящее время требуются работники в самых различных сферах деятельности во многих регионах нашей страны и за ее пределами. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [16]:
query_4_1 = f'''
      SELECT
              a.name AS "Регион",
              COUNT(v.id) AS "Количество вакансий"
        FROM  public.vacancies v
              LEFT JOIN public.areas a 
              ON v.area_id = a.id
       GROUP  BY a.name
       ORDER  BY 2 desc
       LIMIT  5
'''

In [17]:
df = pd.read_sql_query(query_4_1, connection)
display(df)

,Регион,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [18]:
query_4_2 = f'''
      SELECT
              COUNT(id)
        FROM  public.vacancies 
       WHERE  salary_from IS NOT NULL 
              OR salary_to IS NOT NULL
'''

In [19]:
df = pd.read_sql_query(query_4_2, connection)
display(df)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [20]:
query_4_3 = f'''
      SELECT
              ROUND(AVG(salary_from)) AS "Средняя з/п нижняя граница",
              ROUND(AVG(salary_to)) AS " Средняя з/п верхняя граница"
        FROM  public.vacancies
'''

In [21]:
df = pd.read_sql_query(query_4_3, connection)
display(df)

,Средняя з/п нижняя граница,Средняя з/п верхняя граница
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.

In [22]:
query_4_4 = f'''
      SELECT
              schedule,
              employment,
              COUNT(id)
        FROM  public.vacancies
       GROUP  BY 1,2
       ORDER  BY 3 desc
      OFFSET  1 
       LIMIT  1
'''

In [23]:
df = pd.read_sql_query(query_4_4, connection)
display(df)

,schedule,employment,count
0,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [24]:
query_4_5 = f'''
      SELECT
              experience,
              COUNT(experience)
        FROM  public.vacancies
       GROUP  by experience
       ORDER  BY 2
'''

In [25]:
df = pd.read_sql_query(query_4_5, connection)
display(df)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

 ### Выводы:
Проведя детальный анализ данных, выяснили ,что наибольшее количество вакансий находится в городах-столицах (Москва, Санкт Петербург), столицах соседних государств (Минск, Алматы), а также в столице Сибири (Новосибирске). Средняя зарплата по вакансиям сотавляет более 90 т.руб. Наиболее популярным сочетанием рабочего графика и типа трудоустройства является - удаленная работа - полная занятость. Требуется большое количество работников с опытом работы от 1 года до трех лет. Молодые специалисты вполне могут найти интересную работу с достойной оплатой труда.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [26]:
query_5_1 = f'''
     (SELECT
              e.name,      
              COUNT(v.id)
        FROM  public.vacancies v
              LEFT JOIN employers e 
              ON v.employer_id = e.id
       GROUP  BY e.id
       ORDER  BY 2 desc
       LIMIT  1)

       UNION 
             
     (SELECT
              e.name,       
              COUNT(v.id)   
        FROM  public.vacancies v
              LEFT JOIN employers e 
              ON v.employer_id = e.id
       GROUP  BY e.id
       ORDER  BY 2 desc
      OFFSET  4 
       LIMIT  1)
       order  by 2 DESC
'''

In [27]:
df = pd.read_sql_query(query_5_1, connection)
display(df)

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.

In [28]:
query_5_2 = f'''
      SELECT
              a.name,
              COUNT(DISTINCT e.id),
              COUNT(DISTINCT v.id)
        FROM  public.areas a
              LEFT JOIN public.employers e 
              ON e.area = a.id
              
              LEFT JOIN public.vacancies v 
              ON v.area_id = a.id
       WHERE  v.id IS NULL
       GROUP  BY a.name
       ORDER  BY 2 desc
       LIMIT  1
'''

In [29]:
df = pd.read_sql_query(query_5_2, connection)
display(df)

,name,count,count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.

In [30]:
query_5_3 = f'''
      SELECT
              e.name AS name,
              COUNT(distinct v.area_id) AS cnt_area
        FROM  employers e
              JOIN vacancies v 
              ON e.id = v.employer_id
       GROUP  BY e.name
       ORDER  BY cnt_area DESC
       LIMIT  1
        '''

In [31]:
df = pd.read_sql_query(query_5_3, connection)
display(df)

,name,cnt_area
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [32]:
query_5_4 = f'''
      SELECT
              COUNT(e.name) AS cnt_name
        FROM  employers e
              LEFT JOIN employers_industries ei 
              ON e.id = ei.employer_id
              
              LEFT JOIN industries i 
              ON i.id = ei.industry_id
       WHERE  i.name IS NULL
        '''

In [33]:
df = pd.read_sql_query(query_5_4, connection)
display(df)

,cnt_name
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [34]:
query_5_5 = f'''
      SELECT
              e.name,
              COUNT(i.name) AS cnt_name
        FROM  employers e
              LEFT JOIN employers_industries ei 
              ON e.id = ei.employer_id
              
              LEFT JOIN industries i 
              ON i.id = ei.industry_id
       GROUP  BY e.name
      HAVING  COUNT(i.name) = 4
       ORDER  BY e.name
      OFFSET  2 
       LIMIT  1
'''

In [35]:
df = pd.read_sql_query(query_5_5, connection)
display(df)

,name,cnt_name
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.

In [36]:
query_5_6 = f'''
      SELECT
              COUNT(e.name) AS cnt_name
        FROM  employers e
              LEFT JOIN employers_industries ei 
              ON e.id = ei.employer_id
              
              LEFT JOIN industries i 
              ON i.id = ei.industry_id
       WHERE  i.name ='Разработка программного обеспечения'
'''

In [37]:
df = pd.read_sql_query(query_5_6, connection)
display(df)

,cnt_name
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [38]:
df =pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2')
cities = df[1]['Город']
cities = cities.replace('Воронеж[a]','Воронеж')
cities = tuple(cities)
display(cities)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [39]:
query_5_7 = f'''
      SELECT
              a.name AS name,
              COUNT(v.id) AS cnt_vacancies
        FROM  vacancies v
              JOIN areas a 
              ON v.area_id = a.id
              
              JOIN employers e 
              ON v.employer_id = e.id
       WHERE  e.name = 'Яндекс'
              AND a.name IN {cities}
       GROUP  BY a.id

       UNION  ALL

       SELECT
               'Total',
               COUNT(v.id)
        FROM  vacancies v
              JOIN areas a 
              ON v.area_id = a.id
              
              JOIN employers e 
              ON v.employer_id = e.id
       WHERE  e.name = 'Яндекс' 
              AND a.name IN {cities}
       ORDER  BY 2
'''

In [40]:
df = pd.read_sql_query(query_5_7, connection)
display(df)

,name,cnt_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

### Выводы:
Проведя исследования данных о работодателях делаем вывод, что наибольшее количество вакансий предоставляют большие корпорации находящиеся на территории РФ. Основная часть вакансий сконцентрирована в сфере IT - технологий по самым различным сферам. Многие компании предлагают работу в городах-миллионниках.

# Юнит 6. Анализ работодателей

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*

In [41]:
query_6_1= f'''
      SELECT 
              COUNT(name) AS cnt
        FROM  vacancies
       WHERE  name ILIKE '%data%'
              OR name ILIKE '%данн%'
'''

In [42]:
df = pd.read_sql_query(query_6_1, connection)
display(df)

,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.

In [43]:
query_6_2 = f'''
      SELECT 
              COUNT(id)
        FROM  vacancies
       WHERE  (name ILIKE '%data scientist%'
              OR name ILIKE '%data science%'
              OR name ILIKE '%исследователь данных%'
              OR name ILIKE '%machine learning%'
              OR name ILIKE '%машинн%обучен%'
              OR name LIKE '%ML%')
              AND name NOT LIKE '%HTML%'
              AND (name ILIKE '%junior%'
              OR employment = 'Стажировка'
              OR experience = 'Нет опыта')
'''

In [44]:
df = pd.read_sql_query(query_6_2, connection)
display(df)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [45]:
query_6_3 = f'''
      SELECT 
              COUNT(id)
        FROM  vacancies
       WHERE  (name ILIKE '%data scientist%'
              OR name ILIKE '%data science%'
              OR name ILIKE '%исследователь данных%'
              OR name ILIKE '%machine learning%'
              OR name ILIKE '%машинн%обучен%'
              OR name LIKE '%ML%')
              AND name NOT LIKE '%HTML%'
              AND (key_skills ilike '%SQL%'
              OR key_skills = '%postgres%')
'''

In [46]:
df = pd.read_sql_query(query_6_3, connection)
display(df)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [47]:
query_6_4 = f'''
      SELECT 
              COUNT(id)
        FROM  vacancies
       WHERE  (name ILIKE '%data scientist%'
              OR name ILIKE '%data science%'
              OR name ILIKE '%исследователь данных%'
              OR name ILIKE '%machine learning%'
              OR name ILIKE '%машинн%обучен%'
              OR name LIKE '%ML%')
              AND name NOT LIKE '%HTML%'
              AND key_skills ilike '%Python%'
'''

In [48]:
df = pd.read_sql_query(query_6_4, connection)
display(df)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [49]:
query_6_5 = f'''
      SELECT 
              ROUND(AVG(LENGTH(key_skills)-LENGTH(REPLACE(key_skills,'\t',''))+1),2) AS rnd_skill
        FROM  vacancies
       WHERE  (name ILIKE '%data scientist%'
              OR name ILIKE '%data science%'
              OR name ILIKE '%исследователь данных%'
              OR name ILIKE '%machine learning%'
              OR name ILIKE '%машинн%обучен%'
              OR name LIKE '%ML%')
              AND name NOT LIKE '%HTML%'
              AND key_skills IS NOT NULL
'''

In [50]:
df = pd.read_sql_query(query_6_5, connection)
display(df)

,rnd_skill
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [51]:
query_6_6 = f'''
      SELECT 
              experience,
              ROUND(AVG(COALESCE((salary_from+salary_to)/2,salary_from,salary_to))) AS avg_salary
        FROM  vacancies
       WHERE  (name ILIKE '%data scientist%'
              OR name ILIKE '%data science%'
              OR name ILIKE '%исследователь данных%'
              OR name ILIKE '%machine learning%'
              OR name ILIKE '%машинн%обучен%'
              OR name LIKE '%ML%')
              AND name NOT LIKE '%HTML%'
              AND (salary_from IS NOT NULL
              OR salary_to IS NOT NULL)
       GROUP  BY experience
       ORDER  BY avg_salary
'''

In [52]:
df = pd.read_sql_query(query_6_6, connection)
display(df)

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### Выводы:
Предметно изучив информацию делаем вывод,что требуется огромное количество специалистов по работе с данными.
Достаточно большой объем вакансий для начинающих специалистов по Data Science уровня Junior. Причем средний уровень заработной платы таких соискателей гораздо выше, чем средний по стране в других профессиях.

***

#### Дополнительные исследования
Создадим запрос, чтобы узнать  пятерку лидеров по количеству ключевых навыков,которые работодатель требует  в первую очередь.

In [53]:
query_7_1 = f'''
      SELECT 
              SPLIT_PART(key_skills,'\t',1) AS skill,
              COUNT(split_part(key_skills,'\t',1))
        FROM  vacancies 
       GROUP  BY skill
       ORDER  BY 2 desc
       LIMIT  5
'''

In [54]:
df = pd.read_sql_query(query_7_1, connection)
display(df)

,skill,count
0,SQL,1949
1,Python,1773
2,JavaScript,1546
3,Английский язык,1472
4,Git,1336


***

# Общий вывод по проекту

Исходы из вышеизложенного мы можемсделать следующие заключение. Рынок труда довольно огромен. Каждый желающий трудиться может найти работу. Вакансии представлены очень широким спектром сфер деятельности (промышленность, медицина, торговля, народное хозяйство и.т.д.) Наиболее популярным напрвлением в настоящее время являются информационные технологии, а именно Data Science. Из исследований в п. 7.1 мы видим, что первым ключевым навыком из всей массы вакансий работодатель ставит (SQL, Python, JavaScript, Английский язык, GIT). Из пяти первых SkillFactory включил в курс по DataScience три позиции. Значит мы находимся на правильном пути.